# Assignment #2 IRIS 데이터를 이용한 Deep Neural Network 모델 설계

- 202055623
- 허치영

## 라이브러리 import

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torchsummary import summary
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
## pytorch device 설정

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


## Dataset 불러오기

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
iris = fetch_ucirepo(id=53)

In [4]:
# data (as pandas dataframes)
X = iris.data.features
y = iris.data.targets

# metadata
print(iris.metadata)

# variable information
print(iris.variables)

{'uci_id': 53, 'name': 'Iris', 'repository_url': 'https://archive.ics.uci.edu/dataset/53/iris', 'data_url': 'https://archive.ics.uci.edu/static/public/53/data.csv', 'abstract': 'A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.\n', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 150, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1936, 'last_updated': 'Tue Sep 12 2023', 'dataset_doi': '10.24432/C56C76', 'creators': ['R. A. Fisher'], 'intro_paper': {'title': 'The Iris data set: In search of the source of virginica', 'authors': 'A. Unwin, K. Kleinman', 'published_in': 'Significance, 2021', 'year': 2021, 'url': 'https://www.semanticscholar.org/paper/4599862ea877863669a6a8e63a3c707a787d5d7e', 'doi': '1740-9713.01589'}, 'add

## 데이터 전처리

In [5]:
# target이 문자열이므로 LabelEncoder()를 사용하여 숫자로 변환

le = LabelEncoder()
display(y[:5])
y = le.fit_transform(y)
display(y[:5])

,class
0,Iris-setosa
1,Iris-setosa
2,Iris-setosa
3,Iris-setosa
4,Iris-setosa


/Users/chiyeong/Documents/projects/I-am-Learning-Machine-Learning/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, 0, 0])

In [6]:
# Train : Test = 8:2로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# StandardScaler instance
scaler = StandardScaler()

# X_train에 대해 fit() 메서드를 호출하여 평균과 표준편차 계산
scaler.fit(X_train)

# X_train과 X_test data transform
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# scaling 결과 확인
print("X_train mean: ", X_train.mean())
print("X_train std: ", X_train.std())
print("X_test mean: ", X_test.mean())
print("X_test std: ", X_test.std())

X_train mean:  4.357625371653739e-16
X_train std:  1.0
X_test mean:  0.0914914534439311
X_test std:  0.9984293963082908


- `LabelEn₩coder`를 이용하여 target을 숫자로 변환
- `StandardScaler`를 이용하여 feature를 정규화

## MLP 모델 구성

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        # input layer : 4 -> 64
        # dataset의 input feature가 4개이므로 input layer의 노드 수는 4
        self.input_layer = nn.Linear(4, 64)
        # hidden layer : 64 -> 64
        self.hidden_layer0 = nn.Linear(64, 64)
        self.hidden_layer1 = nn.Linear(64, 64)
        # output layer : 64 -> 3
        # dataset의 target class가 3개이므로 output layer의 노드 수는 3
        self.output_layer = nn.Linear(64, 3)
        # activation function : tanh
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = x.to(device)
        x = self.tanh(self.input_layer(x))
        x = self.tanh(self.hidden_layer0(x))
        x = self.tanh(self.hidden_layer1(x))
        x = self.output_layer(x)
        return x

In [8]:
model = MLP().to(device)
print(model)

MLP(
  (input_layer): Linear(in_features=4, out_features=64, bias=True)
  (hidden_layer0): Linear(in_features=64, out_features=64, bias=True)
  (hidden_layer1): Linear(in_features=64, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=3, bias=True)
  (tanh): Tanh()
)


- Pytorch 라이브러리를 이용
- `nn.Module`을 상속받아 모델을 구성

## 모델 학습

In [9]:
# K-Fold Cross Validation 수행
# k = 4 (Train : Val : Test = 6 : 2 : 2)

skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# hyperparameters
learning_rate = 0.01
epochs = 5000

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# training loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold [{fold+1}/{skf.get_n_splits()}]")
    # train set, validation set 분할
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    for epoch in range(epochs):
        model.train()  # 학습 모드로 설정
        optimizer.zero_grad()  # optimizer gradient 초기화
        # Pytorch Tensor로 변환
        X_train_tensor = torch.tensor(X_train_fold, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_fold, dtype=torch.long).to(device)
        # forward pass
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)  # loss 계산
        # backward pass
        loss.backward()
        optimizer.step()  # weight 업데이트

        if epoch % 1000 == 0:
            print(f"Epoch [{epoch}/{epochs}] Train Loss: {loss.item()}")

    # validation loss 계산
    model.eval()
    X_val_tensor = torch.tensor(X_val_fold, dtype=torch.float32).to(device)
    y_val_tensor = torch.tensor(y_val_fold, dtype=torch.long).to(device)
    y_pred = model(X_val_tensor)
    loss = criterion(y_pred, y_val_tensor)
    print(f"Validation Loss: {loss.item()}")

Fold [1/4]
Epoch [0/5000] Train Loss: 1.1143018007278442
Epoch [1000/5000] Train Loss: 0.12158060818910599
Epoch [2000/5000] Train Loss: 0.06887462735176086
Epoch [3000/5000] Train Loss: 0.060344696044921875
Epoch [4000/5000] Train Loss: 0.057450003921985626
Validation Loss: 0.0449000708758831
Fold [2/4]
Epoch [0/5000] Train Loss: 0.04187144711613655
Epoch [1000/5000] Train Loss: 0.033216431736946106
Epoch [2000/5000] Train Loss: 0.028511393815279007
Epoch [3000/5000] Train Loss: 0.024516310542821884
Epoch [4000/5000] Train Loss: 0.020958004519343376
Validation Loss: 0.16881056129932404
Fold [3/4]
Epoch [0/5000] Train Loss: 0.06925053894519806
Epoch [1000/5000] Train Loss: 0.03168404847383499
Epoch [2000/5000] Train Loss: 0.0279790498316288
Epoch [3000/5000] Train Loss: 0.026340648531913757
Epoch [4000/5000] Train Loss: 0.024907706305384636
Validation Loss: 0.2640412151813507
Fold [4/4]
Epoch [0/5000] Train Loss: 0.10093733668327332
Epoch [1000/5000] Train Loss: 0.047181446105241776
Ep

- 4-fold cross validation 진행 (train : val : test = 6 : 2 : 2)
- `StratifiedKFold`를 이용하여 class 분포를 유지
- Loss function : CrossEntropyLoss
- Optimizer : Stoachastic Gradient Descent
- Learning rate : 0.01
- Epoch : 5000

## Test set으로 모델 평가

In [10]:
model.eval() # 평가 모드로 설정
# Pytorch Tensor로 변환
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
y_pred = model(X_test_tensor)
loss = criterion(y_pred, y_test_tensor) # loss 계산

print(f"Test Loss: {loss.item()}")
_, predicted = torch.max(y_pred, 1) # 예측값
correct = (predicted == y_test_tensor).sum().item() # 정답 개수
accuracy = correct / y_test_tensor.size(0) # Categorical Accuracy 계산
print(f"Test Accuracy: {accuracy}")



Test Loss: 0.009720893576741219
Test Accuracy: 1.0


## 모델 결과 출력

In [11]:
summary(model, (4,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             320
              Tanh-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              Tanh-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              Tanh-6                   [-1, 64]               0
            Linear-7                    [-1, 3]             195
Total params: 8,835
Trainable params: 8,835
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.03
Estimated Total Size (MB): 0.04
----------------------------------------------------------------
